In [ ]:
import matplotlib.pyplot as plt
from numpy import array
from pickle import dump
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.callbacks import LambdaCallback, EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
from keras.optimizers import RMSprop
from keras.utils import to_categorical
import tensorflow as tf
import string

In [ ]:
# load text to memory
def load_doc(filename): 
# open the file as read only
    file = open(filename, 'r')
# read all text
    text = file.read()
# close the file
    file.close()
    return text

In [ ]:

##heuristics for splitting up words
#replace dashes with a white space
#otherwise words based on white space.
#reduce all words to lowercase and remove punctuation from words 
#remove all words that are not alphabetic 
# turn a doc into clean tokens



def clean_doc(doc):
# replace '--' with a space ' '
    doc = doc.replace('--', ' ')
# split into tokens by white space
    tokens = doc.split()
# remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
# remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
# make lower case
    tokens = [word.lower() for word in tokens]
    return tokens


In [ ]:
# use GPU to speed up training time through tensforflow
config = tf.ConfigProto()
#only allocate as much GPU memory based on runtime allocations, initially little but allows memory to be extended
config.gpu_options.allow_growth = True


In [ ]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()


In [ ]:
# load document
in_filename = '/Users/neilwatt/Documents/BIs/PrWeb/2018Posts/October/canterburytales.txt'
doc = load_doc(in_filename)
print(doc[:200])

In [ ]:
# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))


In [ ]:
# organize into sequences of tokens
#difference between sequences (previous) and sentences is that I fixed issue
#with repeating words
length = 50 + 1
#move forward 3 words
step = 2
sequences = list()
for i in range(length, len(tokens), step):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

In [ ]:
# save sequences to file
out_filename = 'canterburytales_sequences.txt'

save_doc(sequences, out_filename)

In [ ]:
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
 
# load
in_filename = 'canterburytales_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [20]:
#representation of the model is distributed
#- i.e words with similar meaning have similar representation
#representation learned simulataneously with model
#probability of next word using context of last 100 words
#embedding layer for representation, LSTM to predict words on context

# integer encode sequences of words
#mapping each word to unique integer and encoding input sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

# vocabulary size
#i add a one since vocabulary begins from index one 
#but indexing arrays is zero offset
vocab_size = len(tokenizer.word_index) + 1

In [21]:
tokenizer.word_index

{'cantebrig': 10245,
 'whenever': 9010,
 'waileth': 5456,
 'mood': 6726,
 'aprilis': 13620,
 'commandments': 4864,
 'leyes': 9499,
 'forgave': 13489,
 'ovide': 7818,
 'laud': 3149,
 'sirens': 15809,
 'bedrid': 9054,
 'pomely': 13743,
 'glides': 13526,
 'beg': 3834,
 'graves': 14890,
 'proserpina': 5324,
 'chronicle': 15752,
 'altered': 17236,
 'duke': 777,
 'prosperous': 6704,
 'ferlie': 14347,
 'gestae': 14481,
 'rending': 8900,
 'angelic': 17072,
 'peer': 2093,
 'spiced': 4611,
 'amateurs': 10196,
 'counsellors': 3543,
 'week': 3542,
 'marketplace': 14903,
 'guilt': 1266,
 'clapt': 6819,
 'therewith': 695,
 'dignified': 15641,
 'holes': 6410,
 'potion': 5735,
 'preclude': 12602,
 'workes': 1287,
 'gentlenesse': 13405,
 'lions': 5827,
 'foregoing': 9961,
 'penoncel': 17425,
 'wellfaring': 15562,
 'palmieri': 9649,
 'elate': 15594,
 'parfay': 5008,
 'unwholesome': 17252,
 'energy': 12157,
 'looks': 9408,
 'ashes': 1728,
 'traisen': 17257,
 'admitting': 5726,
 'jewes': 1809,
 'sophistri

In [ ]:

#need to seperate word encodings into  y (output) and x (input)
# separate output from input with array slicing
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [17]:
Embedding

keras.layers.embeddings.Embedding

In [13]:

# defining model
# two LSTM hidden layers with 100 memory cells
#dense fully connected layer with 100 neurons connects to the LSTM hidden layers 
#to interpret the features extracted from the sequence
#output layer predicts the next word as a single vector the size of the vocabulary 
#with a probability for each word in the vocabulary
#softmax activation function is used to ensure 
#the outputs have the characteristics of normalized probabilities
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100))
#model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.5))
#model.add(LSTM(100))
#model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dense(100))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            885650    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 17713)             1789013   
Total params: 2,755,263
Trainable params: 2,755,263
Non-trainable params: 0
_________________________________________________________________


In [14]:

# compile model
optimizer = RMSprop(lr=0.007)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [ ]:
# define the checkpoint so I can load model in future
filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, 
                             monitor='loss', 
                             verbose=1, 
                             save_best_only=True, 
                 
                             mode='min')
earlystop = EarlyStopping(monitor='val_loss', patience=2,
                              verbose=2)

# fit model using the gpu
#batch size is a particularly important hyperparamter which I intend to play around within future posts
with tf.device('/gpu:0'):

    history=model.fit(X, y,validation_split=0.1,batch_size=80,epochs=60,verbose=1,callbacks=[ checkpoint,earlystop])


Train on 119114 samples, validate on 13235 samples
Epoch 1/60
 48320/119114 [===========>..................] - ETA: 5:01 - loss: 7.0144 - acc: 0.0417     

In [ ]:
print(history.history.keys())

In [ ]:
history.history['val_loss']

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [ ]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
 
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
 


In [ ]:
n_words

In [ ]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)
 
# load cleaned text sequences
in_filename = '/Users/neilwatt/Documents/BIs/PrWeb/2018Posts/October/Code/canterburytales_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1
 
# load the model
model = load_model('weights.hdf5')
 
# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))
 
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')
 
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 3)
print(generated)

In [ ]:
seed_text